In [1]:
import pandas as pd
import numpy as np
import myfitnesspal
import datetime


from flask import g, Flask, render_template, request, session, redirect, url_for, jsonify
from werkzeug.wrappers import Request, Response

from werkzeug.serving import run_simple

# MyFitnessPal

In [2]:
client = myfitnesspal.Client('groupb3', password='groupb3thebest')

In [3]:
date = datetime.date.today()
day = client.get_date(date)
print(day)
print(day.exercises[0].get_as_list())

calories = 0
#calories = day.exercises[0].get_as_list()[0]['nutrition_information']['calories burned']
print('You burned ' + str(calories) + ' calories today')

03/18/22 {}
[]
You burned 0 calories today


In [4]:
current_weight = client.get_measurements(lower_bound = datetime.date.today())
print('Your current weight is: ' + str(current_weight) + 'kg')

Your current weight is: OrderedDict()kg


# WebApp

In [59]:
## Stores (prefer get from DB ofcourse later on)
#Create list or dict from multiple stores

lat = 52.356
lng = 4.953
loc = "Science park"

content_string = 'DESCRIPTION OF THIS STORE'

In [60]:
## Hotspots
#Create list or dict for multiple hotspots
lat = 52.356
lng = 4.953
loc = "Science park"

content_string = 'DESCRIPTION OF THIS HOTSPOT'


In [61]:
content_string = 'BLABLABLA'

In [62]:
app = Flask(__name__)


In [63]:
# @app.route("/")
# def index(name=None):
#     return render_template('map.html', lng = lng, lat = lat, content_string = content_string)

@app.route("/")
def index(name=None):
    return render_template('index.html', name = name)

In [64]:
@app.route("/map")
def maps():
    return render_template('map.html', lng = lng, lat = lat, content_string = content_string)

In [65]:
@app.route("/directions")
def directions():
    return render_template('direction_scenic.html', lng = lng, lat = lat, content_string = content_string, loc = loc)

In [66]:
@app.route("/profile", methods = ['POST', 'GET'])
def profile(calories = calories, current_weight = current_weight):
    user_name = request.form.get('exampleInputEmail1')
    print(user_name)
    return render_template('profile.html', calories = calories, current_weight = current_weight)

In [67]:
run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Mar/2022 10:00:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 10:00:48] "GET /stylesheet.css HTTP/1.1" 404 -
127.0.0.1 - - [18/Mar/2022 10:00:48] "GET /static/images/profile.png HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 10:00:48] "GET /static/images/maps.png HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 10:00:48] "GET /static/images/icon.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Mar/2022 10:00:54] "GET /profile HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 10:00:54] "GET /static/images/token_1%20(1).png HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 10:00:54] "GET /static/images/profile.png HTTP/1.1" 304 -


None


127.0.0.1 - - [18/Mar/2022 10:00:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 10:00:56] "GET /stylesheet.css HTTP/1.1" 404 -
127.0.0.1 - - [18/Mar/2022 10:00:56] "GET /static/images/icon.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Mar/2022 10:00:56] "GET /static/images/profile.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Mar/2022 10:00:56] "GET /static/images/maps.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Mar/2022 10:01:00] "GET /directions HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 10:01:00] "GET /stylesheet.css HTTP/1.1" 404 -
127.0.0.1 - - [18/Mar/2022 10:01:00] "GET /static/images/icon.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Mar/2022 10:01:00] "GET /static/images/profile.png HTTP/1.1" 304 -
127.0.0.1 - - [18/Mar/2022 10:01:00] "GET /static/icon.png HTTP/1.1" 404 -
127.0.0.1 - - [18/Mar/2022 10:01:00] "GET /static/icon.png HTTP/1.1" 404 -
127.0.0.1 - - [18/Mar/2022 10:01:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Mar/2022 10:01:07] "GET /stylesheet.css HTTP/1.1" 404 -
127.0.0.1 - - [18/Mar/